In [9]:
from frust.vis import MolTo3DGrid
from rdkit import Chem
from rdkit.Chem import RWMol
from rdkit.Chem.rdchem import BondType
from frust.utils.mols import combine_rw_mols

In [17]:
def transformer_mols(
    ligand_smiles = "C1=CC=CO1",
    catalyst_smiles = "CC1(C)CCCC(C)(C)N1C2=CC=CC=C2B",
    only_uniques = False,
    only_generics = False,
    show_IUPAC = True,
    select: str | list[str] | None = None,
    key_prefix: str | None = None,
):

    # --- prepare input molecules ---
    catalyst_mol = Chem.MolFromSmiles(catalyst_smiles)
    ligand_mol   = Chem.MolFromSmiles(ligand_smiles)
    catalyst_rw  = RWMol(catalyst_mol)
    ligand_rw    = RWMol(ligand_mol)

    ####################
    ### Create dimer ###
    ####################
    catalyst1 = Chem.MolFromSmiles(catalyst_smiles)
    catalyst2 = Chem.MolFromSmiles(catalyst_smiles)
    B_pattern_dimer = Chem.MolFromSmarts("[B]c1ccccc1")
    B_match_dimer = catalyst1.GetSubstructMatches(B_pattern_dimer)
    B_idx_1 = B_match_dimer[0][0]
    catalyst1 = Chem.AddHs(catalyst1, onlyOnAtoms=[B_idx_1])
    catalyst2 = Chem.AddHs(catalyst2, onlyOnAtoms=[B_idx_1])
    catalyst1_RW = RWMol(catalyst1)
    catalyst2_RW = RWMol(catalyst2)
    dimer, offset = combine_rw_mols(catalyst1_RW, catalyst2_RW)
    BHH_pattern = Chem.MolFromSmarts("B([H])([H])c1ccccc1")
    BHH_match = catalyst1.GetSubstructMatches(BHH_pattern)
    cat1_H1_idx, cat1_H2_idx = BHH_match[0][1], BHH_match[0][2]
    cat2_H1_idx = cat1_H1_idx + offset
    cat2_H2_idx = cat1_H2_idx + offset
    cat1_B_idx  = BHH_match[0][0]
    cat2_B_idx  = cat1_B_idx + offset
    dimer.AddBond(cat1_H1_idx, cat2_B_idx, Chem.BondType.SINGLE)
    dimer.AddBond(cat2_H1_idx, cat1_B_idx, Chem.BondType.SINGLE)
    for idx, charge in [(cat1_B_idx, -1), (cat2_B_idx, -1),
                        (cat1_H1_idx, +1), (cat2_H1_idx, +1)]:
        atom = dimer.GetAtomWithIdx(idx)
        atom.SetFormalCharge(charge)
    dimer_mol = dimer.GetMol()
    Chem.SanitizeMol(dimer_mol)

    ####################
    ### Create HH  ###
    ####################
    HH_mol = Chem.MolFromSmiles("[H][H]")

    ######################
    ### Find unique cH ###
    ######################
    cH_patt = Chem.MolFromSmarts('[cH]')
    matches = ligand_rw.GetSubstructMatches(cH_patt)
    cH_atoms = [ind[0] for ind in matches]
    atom_rank = list(Chem.CanonicalRankAtoms(ligand_rw, breakTies=False))
    def find_unique_atoms(lst):
        seen = set(); out = []
        for i, x in enumerate(lst):
            if x not in seen:
                seen.add(x); out.append(i)
        return out
    unique_cH = set(find_unique_atoms(atom_rank)).intersection(cH_atoms)

    ############################################
    ### Create intermediate 2 and molecule 2 ###
    ############################################
    b_pattern = Chem.MolFromSmarts("[B]")

    catalyst_matches = catalyst_mol.GetSubstructMatches(b_pattern)

    if not catalyst_matches:
        raise ValueError("No [B] atom found in the catalyst.")

    catalyst_b_idx = catalyst_matches[0][0]

    mol2s = []
    int2s = []
    for cH in unique_cH:

        combined_rw, offset = combine_rw_mols(catalyst_rw, ligand_rw)
        combined_mol = combined_rw.GetMol()
        Chem.SanitizeMol(combined_mol)
        
        b_idx_combined   = catalyst_b_idx
        ch_idx_combined  = cH + offset

        combined_rw.AddBond(b_idx_combined, ch_idx_combined, Chem.BondType.SINGLE)

        mol2 = combined_rw.GetMol()

        boron = combined_rw.GetAtomWithIdx(catalyst_b_idx)
        boron.SetFormalCharge(-1)

        TMP = Chem.MolFromSmarts('CC1(C)CCCC(C)(C)N1')
        TMP_match = combined_rw.GetSubstructMatches(TMP)
        nitrogen = combined_rw.GetAtomWithIdx(TMP_match[0][9])
        nitrogen.SetFormalCharge(+1)

        int2 = combined_rw.GetMol()
        Chem.SanitizeMol(mol2)
        Chem.SanitizeMol(int2)

        mol2s.append((mol2, ch_idx_combined))
        int2s.append((int2, ch_idx_combined))

    MolTo3DGrid(mol2s[0][0])

transformer_mols()

[14:28:14] Molecule does not have explicit Hs. Consider calling AddHs()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.